In [ ]:
#libraries
import pandas as pd
from datetime import datetime, timedelta
import sys
sys.path.insert(0, "C:/Soft/repos/cwms-python")
import cwms
import numpy as np

## Copy Locations

In [ ]:
apiRoot_src = "https://wm.mvp.ds.usace.army.mil:8243/mvp-data/"

In [ ]:
apiRoot_dev = "https://water.dev.cwbi.us/cwms-data/"

In [ ]:
from getpass import getpass
apiKey = getpass()
apiKey_dev = "apikey " + apiKey

In [ ]:
office = 'MVP'

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_src)#, api_key=apiKey)

In [ ]:
location = cwms.get_locations_catalog(office_id=office)

In [ ]:
locations = location.df

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_dev, api_key=apiKey_dev)

In [ ]:
def default_val(value, default):
    if pd.isna(value) or value == 'Unknown or Not Applicable':
        value = default
    return value

def store_multi_location_df(locations):
    for i,row in locations.iterrows():
        if row['active']:
            loc_json = {
              "office-id": row['office'],  # required
              "name": row['name'],    #required
              "latitude": float(default_val(row['latitude'],'38.0')),  #required
              "longitude": float(default_val(row['longitude'],'-85.0')),  #required
              "active": row['active'],  #required
              "public-name": row['public-name'],
              "long-name": row['long-name'],
              "description": row['description'],
              "timezone-name": default_val(row['time-zone'],'US/CENTRAL'), #required
              "location-type": row['type'], 
              "location-kind": row['kind'],  #required
              "nation": 'US',   #required and abbreviated
              #"state-initial": row['state'],  #Saving state doesn't work.
              #"county-name": row['county'],
              "nearest-city": row['nearest-city'],
              "horizontal-datum": default_val(row['horizontal-datum'],'NAD27'),  #required
              "published-longitude": float(row['published-longitude']),
              "published-latitude": float(row['published-latitude']),
              "vertical-datum": row['vertical-datum'],
              "elevation": float(row['elevation']),
              "map-label": row['map-label'],
              "bounding-office-id": row['bounding-office'],
              "elevation-units": row['unit']
            }
            #clean_dict = filter(lambda k: not pd.isna(loc_json[k]), loc_json)
            clean_dict = {k: loc_json[k] for k in loc_json if not pd.isna(loc_json[k])}
            #print(clean_dict)
            try:
                cwms.store_location(data = clean_dict)
            except:
                print(clean_dict)
                print('save failed')

In [ ]:
store_multi_location_df(locations)

In [ ]:
location_dev = cwms.get_locations_catalog(office_id=office)

## Copy TimeSeries Identifiers

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_src)#, api_key=apiKey)

In [ ]:
ts_indentifier = cwms.get_timeseries_identifiers(office_id=office,page_size=100000)

In [ ]:
location = cwms.get_locations_catalog(office_id=office)

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_dev, api_key=apiKey_dev)

In [ ]:
ts_ids = ts_indentifier.df
ts_ids[['location-id','param','type','int','dur','ver']] = ts_ids['time-series-id'].str.split('.', expand=True)
locs = location.df.rename(columns = {'name': 'location-id','office':'office-id'})

In [ ]:

ts_lo_ids = pd.merge(ts_ids, locs, how='inner',on=['location-id','office-id'])

In [ ]:
for i,row in ts_lo_ids.iterrows():
    
    t_id = {
      "office-id": row['office-id'],
      "time-series-id": row['time-series-id'],
      "timezone-name": default_val(row['timezone-name'],'US/Eastern'),
      "interval-offset-minutes": float(row['interval-offset-minutes']),
      "active": row['active_x']
    }
    try:
        cwms.store_timeseries_identifier(data=t_id,fail_if_exists=False)
    except:
        print(row['time-series-id'])
    

In [ ]:
ts_indentifier_dev = cwms.get_timeseries_identifiers(office_id=office,page_size=20000)

In [ ]:
ts_dev = ts_indentifier_dev.df

In [ ]:
merged_df = pd.merge(ts_dev, ts_lo_ids, on='time-series-id', how='outer', indicator=True)

In [ ]:
ts_indentifier.df

## Copy Timeseries group

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_src)#, api_key=apiKey)

In [ ]:
loc_groups = cwms.get_timeseries_group(group_id="USGS TS Data Acquisition",category_id="Data Acquisition",office_id="MVP",category_office_id="CWMS",group_office_id="CWMS")
#loc_groups = cwms.get_timeseries_group(group_id="SHEF Data Acquisition",category_id="Data Acquisition",office_id="LRL",category_office_id="CWMS",group_office_id="CWMS")

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_dev, api_key=apiKey_dev)

In [ ]:
loc_groups.df

In [ ]:
cwms.update_timeseries_groups(data=loc_groups.json,group_id="USGS TS Data Acquisition",office_id=office,replace_assigned_ts=False)

In [ ]:
loc_group_dev = cwms.get_timeseries_group(group_id="USGS TS Data Acquisition",category_id="Data Acquisition",office_id=office,category_office_id="CWMS",group_office_id="CWMS")

In [ ]:
loc_group_dev.df

## Location Groups

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_src)#, api_key=apiKey)

In [ ]:
Loc_group = cwms.get_location_group(loc_group_id="USGS Station Number",category_id="Agency Aliases",office_id="CWMS")

In [ ]:
Loc_group.json

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_dev, api_key=apiKey_dev)

In [ ]:
cwms.update_location_group(data=Loc_group.json, group_id="USGS Station Number",office_id=office,replace_assigned_locs=False)

In [ ]:
Loc_group_dev = cwms.get_location_group(loc_group_id="USGS Station Number",category_id="Agency Aliases",office_id="CWMS")

In [ ]:
Loc_group_dev.df

In [ ]:
Loc_group.df

## Copy Rating Templates

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_src)#, api_key=apiKey)

In [ ]:
templates = cwms.get_rating_templates(office_id=office)

In [ ]:
temps = templates.df

In [ ]:
def rating_template_df_to_xml(data: pd.DataFrame) -> str:
    
    office_id = data['office-id']
    parameters, template_ver = data['id'].split('.')
    dep_param = data['dependent-parameter']
    discription = data['description']
    ind_param_specs = data['independent-parameter-specs']

    template_xml = f'''<?xml version="1.0" encoding="utf-8"?>
    <rating-template office-id="{office_id}">
    <parameters-id>{parameters}</parameters-id>
    <version>{template_ver}</version>
      <ind-parameter-specs>'''

    for i, d in enumerate(ind_param_specs):
        param = d['parameter']
        in_range_meth = d['in-range-method']
        out_range_low_meth = d['out-range-low-method']
        out_range_high_meth = d['out-range-high-method']
        ind_param_xml = f'''\n         <ind-parameter-spec position="{i+1}">
            <parameter>{param}</parameter>
            <in-range-method>{in_range_meth}</in-range-method>
            <out-range-low-method>{out_range_low_meth}</out-range-low-method>
            <out-range-high-method>{out_range_high_meth}</out-range-high-method>
         </ind-parameter-spec>'''
        template_xml = template_xml + ind_param_xml

    template_xml_end = f'''\n      </ind-parameter-specs>
    <dep-parameter>{dep_param}</dep-parameter>
    <description>{discription}</description>
    </rating-template>'''
    template_xml = template_xml + template_xml_end
    return template_xml
    

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_dev, api_key=apiKey_dev)

In [ ]:
for i,row in temps.iterrows():
    temp_xml = rating_template_df_to_xml(row)
    cwms.store_rating_template(data = temp_xml)

    

In [ ]:
templates_dev = cwms.get_rating_templates(office_id=office)

In [ ]:
templates_dev.df

In [ ]:
temps

## copy Rating Specs

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_src)#, api_key=apiKey)

In [ ]:
ratingspecs = cwms.get_rating_specs(office_id=office)

In [ ]:
specs = ratingspecs.df

In [ ]:
api = cwms.api.init_session(api_root=apiRoot_dev, api_key=apiKey_dev)

In [ ]:
for i,row in specs.iterrows():
    temp = pd.DataFrame([row]).reset_index(drop = True).fillna('')
    #print(pd.DataFrame([row]))
    #print(temp)
    #print(temp.loc[0,'office-id'])
    spec_xml = cwms.rating_spec_df_to_xml(data=temp)
    try:
        cwms.store_rating_spec(data=spec_xml, fail_if_exists=False)
    except:
        print(spec_xml)

In [ ]:
ratingspecs_dev = cwms.get_rating_specs(office_id=office)

In [ ]:
rating_specs_dev = ratingspecs_dev.df
rating_specs_dev[rating_specs_dev['source-agency']=='USGS']